# Script to translate SQuAD to French with PyTorch Hub models

## Imports

In [34]:
#!pip install numpy
#!pip install torch
#!pip install transformers
#!pip install --upgrade pandas
#!pip install nltk
#!pip install git+https://github.com/Maluuba/nlg-eval.git@master
#!nlg-eval --setup
#!pip install fastBPE regex requests sacremoses subword_nmt

In [35]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

from transformers_utils import load_json_QuAD_v1

import pandas as pd
import torch
import tqdm 
import json

import regex as re
import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
import difflib

pd.set_option('display.max_colwidth', None)

## Loading Data

In [37]:
df_squad = load_json_QuAD_v1(
    "../SQuAD-data/dev-v1.1.json"
)
df_squad.head()

,id_question,question,answer_span,answer_span_start,id_context,context,doc_title
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bowl 50?,Denver Broncos,177,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bowl 50?,Carolina Panthers,249,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,"Santa Clara, California",403,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Denver Broncos,177,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anniversary of the Super Bowl?,gold,488,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) c

## Fairseq model

In [38]:
en2fr = torch.hub.load(
    'pytorch/fairseq', 
    'transformer.wmt14.en-fr',
    tokenizer='moses', 
    bpe='subword_nmt', 
    #max_target_positions = 2048, 
    #max_input_positions = 2048,
    #decoder_input_dim = 2048,
    #decoder_output_dim = 2048,
    #max_source_positions=2048,
    )

# Use the GPU (optional):
en2fr.cuda()
#en2fr.args.max_target_positions = 2048
#en2fr.args.max_input_positions = 2048

print(en2fr.args)

Using cache found in /home/azureuser/.cache/torch/hub/pytorch_fairseq_master


Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_vaswani_wmt_en_de_big', attention_dropout=0.0, bpe='subword_nmt', bpe_codes='/home/azureuser/.cache/torch/hub/pytorch_fairseq/53f403ba27ab138b06c1a8d78f5bb4f1722567ac3d3b3e41f821ec2cae2974da.7ef8ab763efda16d3c82dd8b5a574bdfe524e078bac7b444ea1a9c5d355b55ae/bpecodes', bpe_separator='@@', clip_norm=0.0, criterion='label_smoothed_cross_entropy', cross_self_attention=False, data='/home/azureuser/.cache/torch/hub/pytorch_fairseq/53f403ba27ab138b06c1a8d78f5bb4f1722567ac3d3b3e41f821ec2cae2974da.7ef8ab763efda16d3c82dd8b5a574bdfe524e078bac7b444ea1a9c5d355b55ae', decoder_attention_heads=16, decoder_embed_dim=1024, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1024, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_bef

In [39]:
def translate_torch(
    model, 
    df, 
    batch_size, 
):
    questions, contexts, answers, titles, context_ids, question_ids = [], [], [], [], [], []
    batch_q, batch_a, batch_c, batch_c_bis = [], [], [], []
    new_context_history, division_history = [], []
    curr_context = 0
    max_shape = df.shape[0]
    for line, iterrow in enumerate(tqdm.tqdm(df.iterrows())):
        row = iterrow[1]
        titles.append(row["doc_title"])
        context_ids.append(row["id_context"])
        question_ids.append(row["id_question"])
        # we avoid translating 2 times the same context
        if len(batch_q)==0:
            curr_context, new_context = row["id_context"], True #id
        else:
            #new_context = (curr_context[:50] != row["context"][:50])
            new_context = (curr_context != row["id_context"])
        #print(curr_context)
        if new_context:
            curr_context = row["id_context"]
            new_context_history.append(len(batch_q))
            input_context = row["context"]
            
            bin_context = model.binarize(model.apply_bpe(model.tokenize(input_context)))
            # if the sentence is too long, we split it into two parts.
            if bin_context.shape[0] > 200:
                sentences = input_context.split(". ")
                separator = len(sentences)//2 + 1
                part1 = model.binarize(model.apply_bpe(model.tokenize(". ".join(sentences[:separator]))))
                part2 = model.binarize(model.apply_bpe(model.tokenize(". ".join(sentences[separator:]))))
                batch_c.append(part1)
                batch_c_bis.append(part2)
                # we note that this context was split
                division_history.append(len(batch_q))
            else: 
                batch_c.append(bin_context)
            
        input_question = row["question"] 
        input_answer = row["answer_span"]
        batch_q.append(model.binarize(model.apply_bpe(model.tokenize(input_question))))
        batch_a.append(model.binarize(model.apply_bpe(model.tokenize(input_answer))))
        
        end = line == max_shape - 1
        if len(batch_q) == batch_size or end:
            generated_questions = model.generate(batch_q, beam=1) #, sampling=False, sampling_topk=1)
            generated_answers = model.generate(batch_a, beam=1) #, sampling=False, sampling_topk=1)
            generated_contexts = model.generate(batch_c, beam=1) #, sampling=False, sampling_topk=1)   
            generated_contexts_bis = model.generate(batch_c_bis, beam=1) #, sampling=False, sampling_topk=1) 
            
            batch_q, batch_a, batch_c, batch_c_bis = [], [], [], []
            
            temp_context = []
            
            index, index_split, completed = 0, -1, False
            _range = max_shape%batch_size if end else batch_size
            for i in range(_range): 
                if not completed:
                    if i >= new_context_history[index]:
                        splited = False
                        # if the context was splited:
                        if new_context_history[index] in division_history:
                            splited = True
                            index_split += 1
                        index_context = index
                        index += 1
                        if index == len(new_context_history): completed=True
                questions.append(model.detokenize(model.remove_bpe(model.string(generated_questions[i][0]['tokens'])))) 
                context = model.detokenize(model.remove_bpe(model.string(generated_contexts[index_context][0]['tokens']))) +\
                    (" " + model.detokenize(model.remove_bpe(model.string(generated_contexts_bis[index_split][0]['tokens']))) if splited else "")
                contexts.append(context)
                answers.append(model.detokenize(model.remove_bpe(model.string(generated_answers[i][0]['tokens']))))
            new_context_history = []
            division_history = []
    translated = pd.DataFrame({
        "id_question": question_ids,
        "question": questions, 
        "answer": answers, 
        "id_context": context_ids,
        "context":contexts, 
        "title": titles,
        })
    return translated

In [40]:
df_new = translate_torch(en2fr, df_squad, 32)
#df_new.to_csv("../FQuAD-data/translated-train.csv", sep=",")

10570it [43:24,  4.06it/s]


In [41]:
df_squad.head(20)

,id_question,question,answer_span,answer_span_start,id_context,context,doc_title
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bowl 50?,Denver Broncos,177,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bowl 50?,Carolina Panthers,249,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,"Santa Clara, California",403,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Denver Broncos,177,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anniversary of the Super Bowl?,gold,488,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) c

In [42]:
df_new.to_csv("../FQuAD-data/translated-valid.csv", sep=",")
df_new.head(20)

,id_question,question,answer,id_context,context,title
0,56be4db0acb8001400a502ec,Quelle équipe de la NFL représentait le CFA au Super Bowl 50 ?,Denver Broncos 2.,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50
1,56be4db0acb8001400a502ed,Quelle équipe de la NFL représentait le NFC au Super Bowl 50 ?,Carolina Panthers,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50
2,56be4db0acb8001400a502ee,Où s'est déroulé le Super Bowl 50 ?,Santa Clara (Californie),0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50
3,56be4db0acb8001400a502ef,Quelle équipe de la NFL a remporté le Super Bowl 50 ?,Denver Broncos 2.,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50
4,56be4db0acb8001400a502f0,Quelle couleur a été utilisée pour souligner le 50e anniversaire du Super Bowl ?,or,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspe

## Extract answer span

In [43]:
df_new = pd.read_csv("../FQuAD-data/translated-valid.csv")

In [44]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

#list(find_all('spam spam spam spam', 'spam'))

def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace("( ", "(").replace(" )", ")")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace("' ", "'").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()

def strip_string(string_):
    # strip start
    if len(string_) == 0: return string_
    alphanum, possible = False, True
    index = -1
    while alphanum == False and possible:
        index += 1
        alphanum = string_[index].isalnum()
        possible = index < len(string_) - 1
    string_ = string_[index:]
    alphanum = False
    index = 0
    while alphanum == False and possible:
        index += 1
        alphanum = string_[-index].isalnum()
        possible = index < len(string_)
    string_ = string_[:-index+1] if index > 1 else string_
    return string_

In [45]:
def extract_answer_span(df_new, df_old):
    """
    This function will search for the position of the text span inside the context.
    If the words are exactly inside the context, it will be easy, if not, we will 
    replace the text span by a span of the same length close to the translated answer,
    among all the texts.
    INPUTS:
    - df_new: pandas dataframe, translated squad
    - df_old: pandas dataframe, original squad.
    OUTPUTS:
    - starts: list, answer_span_start
    - to_drop: list, indexes of rows to delete.
    """
    count = 0
    starts = []
    to_drop = []
    for i, iterrow in tqdm.tqdm(enumerate(df_new.iterrows())):
        dropping = False
        row_old = df_old.iloc[i]
        answer_old = row_old["answer_span"]
        context_old = row_old["context"]
        
        row = iterrow[1]
        try: 
            answer = strip_string(row["answer"])
        except:
            to_drop.append(i)
            dropping = True
            continue
            
        context = row["context"]
        # let's try to find it in french
        positions = list(find_all(context, answer))
        if positions != []:
            if len(positions) == 1:
                start = positions[0]
            if len(positions) > 1:
                positions_old = list(find_all(context_old, answer_old))
                start_old = row_old["answer_span_start"]
                index = positions_old.index(start_old)
                if len(positions_old) == len(positions):
                    start = positions[index]
                else:
                    #start = elem in positions closest to start_old
                    start = positions[np.argmin([abs(position-start_old) for position in positions])]
        else:
            # let's try to find it in english
            positions_bis = list(find_all(context, answer_old)) 
            if len(positions_bis) == 1:
                start = positions_bis[0]
                
            if len(positions_bis) > 1:
                positions_old = list(find_all(context_old, answer_old))
                start_old = row_old["answer_span_start"]
                index = positions_old.index(start_old)
                if len(positions_old) == len(positions_bis):
                    start = positions_bis[index]
                else:
                    #start = elem in positions closest to start_old
                    start = positions_bis[np.argmin([abs(position-start_old) for position in positions_bis])]
            
            if len(positions_bis)==0:
                #compare the scores in english and in french.
                
                answer_tokenized = nltk.word_tokenize(answer)
                answer_old_tokenized = nltk.word_tokenize(answer_old)
                context_tokenized = nltk.word_tokenize(context)
                len_answer = len(answer_tokenized)
                len_answer_old = len(answer_old_tokenized)
                len_context = len(context_tokenized)
                # with difflib
                all_text_spans = [untokenize(context_tokenized[k:k + len_answer]) for k in range(len_context - len_answer + 1)]
                french_match = difflib.get_close_matches(answer, all_text_spans)
                french_similarity_found = len(french_match) > 0
                
                all_text_spans_en = [untokenize(context_tokenized[k:k + len_answer_old]) for k in range(len_context - len_answer_old + 1)]
                english_match = difflib.get_close_matches(answer_old, all_text_spans_en)
                english_similarity_found = len(english_match) > 0
                
                score_french, score_english = 0, 0
                if french_similarity_found or english_similarity_found:
                    if french_similarity_found:
                        score_french = difflib.SequenceMatcher(None, answer, french_match[0]).ratio()
                    if english_similarity_found:
                        score_english = difflib.SequenceMatcher(None, answer_old, english_match[0]).ratio()
                    text_span = french_match[0] if score_french >= score_english else english_match[0]
                    text_span = strip_string(text_span)
                    try:
                        start = list(find_all(untokenize(context_tokenized), text_span))[0]
                    except:
                        to_drop.append(i)
                        dropping = True

                # with BLEU
                #all_text_spans = [context_tokenized[k:k + len_answer] for k in range(len_context - len_answer + 1)]
                #scores = []
                #similarity_found = False
                #for hypothesis in all_text_spans:
                    #score = sentence_bleu(answer_tokenized, hypothesis)
                    #if score > 0:
                    #    similarity_found = True
                    #scores.append(score)
                #if similarity_found:
                #    index = np.argmax(scores)
                #    text_span = untokenize(context_tokenized[index:index + len_answer])
                #    start = list(find_all(context, text_span))[0]

                else:
                    # soit on laisse inchangé... mauvaise idée... soit on prend le text span à la position
                    # de l'anglais, soit on supprime celui là. 
                    #text_span = untokenize(nltk.word_tokenize(context[start_old:])[:len(answer_tokenized)])
                    #start = list(find_all(context, text_span))[0]
                    to_drop.append(i)
                    dropping = True
        if not dropping:
            starts.append(start)
    return starts, to_drop
span_starts, to_drop = extract_answer_span(df_new, df_squad)
df_bis = df_new.drop(to_drop)
df_bis = df_bis.reset_index()

10570it [01:58, 89.46it/s] 


In [46]:
df_bis["answer_span_start"] = span_starts

In [47]:
df_bis = df_bis.rename(columns={"answer": "answer_span"})

In [48]:
try:
    df_bis = df_bis.drop(columns=["index"])
except:
    pass
try:
    df_bis = df_bis.drop(columns=["Unnamed: 0"])
except:
    pass
try:
    df_bis = df_bis.drop(columns=["Unnamed: 0.1"])
except:
    pass

In [49]:
df_bis.head(20)

,id_question,question,answer_span,id_context,context,title,answer_span_start
0,56be4db0acb8001400a502ec,Quelle équipe de la NFL représentait le CFA au Super Bowl 50 ?,Denver Broncos 2.,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50,52
1,56be4db0acb8001400a502ed,Quelle équipe de la NFL représentait le NFC au Super Bowl 50 ?,Carolina Panthers,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50,130
2,56be4db0acb8001400a502ee,Où s'est déroulé le Super Bowl 50 ?,Santa Clara (Californie),0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50,315
3,56be4db0acb8001400a502ef,Quelle équipe de la NFL a remporté le Super Bowl 50 ?,Denver Broncos 2.,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50,52
4,56be4db0acb8001400a502f0,Quelle couleur a été utilisée pour souligner le 50e anniversaire du Super Bowl ?,or,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initia

In [50]:
def post_proc(df):
    df_copy = df.copy()
    answers_bis = []
    end_chars = [".", ",", ":", ";", " ", "-", "'", '"', ")", "`"]
    for i, row in df.iterrows():
        answer = strip_string(row["answer_span"])
        context = row["context"]
        answer_span_start = row["answer_span_start"]
        len_max = len(context)
        stop = False
        j = 0
        while not stop and len(answer) + answer_span_start + j < len_max:
            stop = context[len(answer) + answer_span_start + j] in end_chars
            j += 1
            
        new_answer_span = context[answer_span_start:len(answer) + answer_span_start + j-1]
            
        answers_bis.append(new_answer_span)
    df_copy["answer_span_bis"] = answers_bis
    return df_copy
df_bis = post_proc(df_bis)

In [51]:
df_bis.head(40)


,id_question,question,answer_span,id_context,context,title,answer_span_start,answer_span_bis
0,56be4db0acb8001400a502ec,Quelle équipe de la NFL représentait le CFA au Super Bowl 50 ?,Denver Broncos 2.,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50,52,Denver Broncos a
1,56be4db0acb8001400a502ed,Quelle équipe de la NFL représentait le NFC au Super Bowl 50 ?,Carolina Panthers,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50,130,Carolina Panthers
2,56be4db0acb8001400a502ee,Où s'est déroulé le Super Bowl 50 ?,Santa Clara (Californie),0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50,315,"Clara, en Californie. Comme"
3,56be4db0acb8001400a502ef,Quelle équipe de la NFL a remporté le Super Bowl 50 ?,Denver Broncos 2.,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50,52,Denver Broncos a
4,56be4db0acb8001400a502f0,Quelle couleur a été utilisée pour souligner le 50e anniversaire du Super Bowl ?,or,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il 

In [52]:
df_bis.to_csv("../FQuAD-data/translated-valid_v1.1.csv", sep=",")

## Convert To Json

In [53]:
df_bis = pd.read_csv("../FQuAD-data/translated-valid_v1.1.csv", sep=",")

In [54]:
def dataframe_to_json(dataframe, json_path):
    data = []
    curr_context_id, curr_title = 0, dataframe.loc[0, "title"]
    curr_context = dataframe.loc[0, "context"]
    qas, paragraphs = [], []
    for iterrow in dataframe.iterrows():
        row = iterrow[1]
        answers = [{"answer_start": row["answer_span_start"], "text": row["answer_span_bis"]}]
        if curr_context_id == row["id_context"]:
            qas.append({"answers": answers, "question": row["question"], 'id': row['id_question']})
        else:
            paragraphs.append({"context": curr_context, "qas": qas})
            curr_context_id = row["id_context"]
            curr_context = row["context"]
            qas = [{"answers": answers, "question": row["question"], 'id': row['id_question']}]
        if curr_title != row["title"]:
            data.append({"title": curr_title, "paragraphs": paragraphs})
            paragraphs = []
            curr_title = row['title']
    data.append({"title": curr_title, "paragraphs": paragraphs}) 
    json_structure = {'version': 1, 'data': data}
    with open(json_path, 'w') as fp:
       json.dump(json_structure, fp)
    return None
dataframe_to_json(df_bis, "../FQuAD-data/translated-valid.json")

In [55]:
from transformers_utils import load_json_QuAD_v1
df = load_json_QuAD_v1("../FQuAD-data/translated-valid.json")
df.head(10)

,id_question,question,answer_span,answer_span_start,id_context,context,doc_title
0,56be4db0acb8001400a502ec,Quelle équipe de la NFL représentait le CFA au Super Bowl 50 ?,Denver Broncos a,52,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50
1,56be4db0acb8001400a502ed,Quelle équipe de la NFL représentait le NFC au Super Bowl 50 ?,Carolina Panthers,130,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50
2,56be4db0acb8001400a502ee,Où s'est déroulé le Super Bowl 50 ?,"Clara, en Californie. Comme",315,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50
3,56be4db0acb8001400a502ef,Quelle équipe de la NFL a remporté le Super Bowl 50 ?,Denver Broncos a,52,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",Super_Bowl_50
4,56be4db0acb8001400a502f0,Quelle couleur a été utilisée pour souligner le 50e anniversaire du Super Bowl ?,orairement,487,0,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" 